In [1]:
import gurobipy as grb
from gurobipy import GRB
import scipy.sparse as spr
import numpy as np
import random
import matplotlib.pyplot as plt
#from sympy import symbols, Rational
from IPython.display import display, Math, Markdown
import numpy.ma as ma


In [2]:
class OneToOneITU():
    def __init__(self, n, m, size_params, random_seed, lbs=(1, 1), tol=1e-9):
        self.n = n
        self.m = m
        self.lb_U, self.lb_V = lbs
        self.tol = tol 

        # generate problem
        np.random.seed(random_seed)




        self.A_ij = np.random.randint(1, size_params[0], size=[n, m])/  np.random.randint(1, size_params[0], size=[n, m])
        # values_A = np.array([1,2,4,5,8,10,16,20])
        # self.A_ij = np.random.choice(values_A[size_params[0]],size= [n,m])
        self.B_ij = np.random.randint(0, size_params[1], size=[n, m])

    def D_ij(self, U_i, V_j):
        return U_i[:, None] + self.A_ij * V_j[None, :] - self.B_ij

    def get_U_ij(self, V_j, i_idx ,j_idx = None):
        if j_idx is None:
            return self.B_ij[i_idx] - self.A_ij[i_idx] * V_j[None, :]
        else:
            return self.B_ij[i_idx,j_idx] - self.A_ij[i_idx,j_idx] * V_j[None, j_idx]

    def get_V_ij(self,i_idx, j_idx , U_i):
        return (self.B_ij[i_idx,j_idx] -  U_i)/ self.A_ij[i_idx,j_idx]



#### Checking modules

The following functions check equilibrium conditions for a candidate $(\mu, U, V).$. Below I will refer to the equilibrium conditions as follows:
* (*Primal Feasibility*) $\sum_{i} \mu_{ij} \leq 1$ and $\sum_{j} \mu_{ij} \leq 1 $ (and $\mu_{ij} \in \{0,1\}$),
* (*Primal Complementary Slackness*) $D_{ij}(U_i,V_j) = 0$ if $ \mu_{ij}  = 1$, 
* (*Dual Feasibility*) $D_{ij}(U_i,V_j) \geq 0$,
* (*Dual Complementary Slackness*) $U_i =  U_0 $ if $\sum_j \mu_{ij} = 0$ and $V_i =  V_0 $ if $\sum_i \mu_{ij} = 0,$ 
* (*Individual Rationality*) $U_i \geq  U_0 ,$ $V_i \geq  V_0 .$ 

**Dual Complementary Slackness**: $\forall i,j, \ U_i > U_{0} \implies \sum_j \mu_{ij} = 0, \ V_j > V_{0} \implies \sum_i \mu_{ij} = 0$

In [3]:
def check_dual_CS(self,U_i,V_j,mu_ij, output = False):

    D_CS_i = np.mean((U_i - self.lb_U)* (np.sum(mu_ij,axis=1) - 1))
    D_CS_j = np.mean((V_j - self.lb_V) * (np.sum(mu_ij,axis=0) - 1)) 
    display(Math(fr'\frac{1}{{n}} \sum_{{i}} (U_{{i}} - U_{{lb}} ) * (\sum_{{j}} \mu_{{ij}} -1) = {D_CS_i}'))
    display(Math(fr'\frac{1}{{m}}\sum_{{j}} (V_{{j}} - V_{{lb}} ) * (\sum_{{i}} \mu_{{ij}} -1) = {D_CS_j}'))
    
    if output is True:
        D_CS_i_bool = (U_i - self.lb_U)* (np.sum(mu_ij,axis=1) - 1) >= 0 
        D_CS_j_bool = (V_j - self.lb_V) * (np.sum(mu_ij,axis=0) - 1) >= 0
        return D_CS_i_bool, D_CS_j_bool
    
OneToOneITU.check_dual_CS = check_dual_CS

**Dual Feasibility**:  $\forall i,j,  \ A_{ij} U_i + G_{ij} V_j \geq B_{ij},$

In [4]:
def check_dual_feas(self,U_i,V_j, output = None):
 
    D_feas = np.minimum(self.D_ij(U_i,V_j),0)
    if output:
        return D_feas, np.where(D_feas<0 )
    display(Math(fr'\frac{1}{{nm}} \sum_{{ij}} \min( A_{{ij}} U_i + G_{{ij}} V_j -  B_{{ij}} , 0)  = {D_feas.mean()}'))
    
   
OneToOneITU.check_dual_feas = check_dual_feas 

**Primal Complementary Slackness**:  $\forall i,j, \  \mu_{ij} * (A_{ij} U_i + G_{ij} V_j - B_{ij}) = 0 $

In [5]:
def check_primal_CS(self, U_i,V_j,mu_ij, output = None):

    P_CS_ij = self.D_ij(U_i,V_j) * mu_ij
    
    if output:
        return P_CS_ij
    else:
        display(Math(fr'\frac{1}{{nm}} \sum_{{ij}} \mu_{{ij}} * (A_{{ij}} U_i + G_{{ij}} V_j - B_{{ij}}) = {P_CS_ij.mean()}'))

OneToOneITU.check_primal_CS = check_primal_CS

**Primal Feasibility**:  $\forall i,j, \  \sum_j  \mu_{ij} \leq 1, \ \sum_i  \mu_{ij} \leq 1$

In [6]:
def check_primal_feas(self, mu_ij):

    display(Math(fr'\forall i, \ \sum_{{j}}  \mu_{{ij}} \leq 1: \ { np.all(np.sum(mu_ij,axis=1) <= np.ones(self.n))}'))
    display(Math(fr'\forall j, \ \sum_{{i}}  \mu_{{ij}} \leq 1: \ {np.all(np.sum(mu_ij,axis=0) <= np.ones(self.m)) }'))
    display(Markdown(f"#matched: {int(np.sum(mu_ij))} over {np.minimum(self.n,self.m)}"))

OneToOneITU.check_primal_feas = check_primal_feas

In [7]:
def check_IR(self, U_i,V_j):
    display(Math(fr'\min_{{i}} U_{{i}} = {np.min(U_i).round(3)} \geq {self.lb_U} = U_{{lb}}'))
    display(Math(fr'\min_{{j}} V_{{j}} = {np.min(V_j).round(3)} \geq  {self.lb_V} = V_{{lb}}'))

OneToOneITU.check_IR = check_IR

The following checks all conditions and prints the results

In [8]:
def check_all(self,eq):
    mu_ij, U_i, V_j = eq 
    header_size = 1

    display(Markdown(f"{'_' * 4}\n<h{header_size}>Feasibility</h{header_size}>"))
    self.check_primal_feas(mu_ij)

    display(Markdown(f"{'_' * 4}\n<h{header_size}>Generalized Complementary Slackness</h{header_size}>"))
    self.check_dual_feas(U_i, V_j )
    self.check_primal_CS(U_i, V_j , mu_ij)
    
    display(Markdown(f"{'_' * 4}\n<h{header_size}>Individual Rationality</h{header_size}>"))
    self.check_IR(U_i, V_j )
    self.check_dual_CS(U_i, V_j , mu_ij)

OneToOneITU.check_all = check_all

# Parallel auction

### Algorithm

In [21]:
def ITU_auction(self, max_iter, tol, initial_data = None):

    V_j = np.ones(self.m) * self.lb_V
    mu_ij = np.zeros([self.n,self.m+1], dtype= bool)
    if initial_data is not None:
        V_j = initial_data[1]
        mu_ij = initial_data[0]

    unassigned = mu_ij.sum(1) == 0
    n_unassigned = unassigned.sum()
    iter = 0

    while n_unassigned > 0 and iter < max_iter:
      
        U_ij = np.concatenate((self.get_U_ij(V_j, unassigned), np.ones((n_unassigned,1)) * self.lb_U), axis = 1) 
        ### compute j_star and second best
        sorted_w_id = np.argsort(U_ij , axis = 1)[:,-2:]
        second_best = U_ij[np.arange(n_unassigned),sorted_w_id[:,0] ]
        j_star = sorted_w_id[:,1]
        # clear quitters
        perm_unmatched = (j_star == self.m )
        mu_ij[unassigned,:] *= ~perm_unmatched[:,None]
        mu_ij[unassigned,-1] += perm_unmatched
      
        j_star = j_star[~perm_unmatched]
        second_best = second_best[~perm_unmatched]
        if len(j_star) == 0 :
            break
        
        ### collect offers and pick j's favoutive
        i_offering = np.where(unassigned * (mu_ij[:,-1] == 0) >0)[0]
        offers = self.get_V_ij(i_offering, j_star,  second_best) 
        j_star_idx = np.unique(j_star)
        offering_ind = j_star[:,None] == j_star_idx
        masked_matrix = ma.masked_array(np.tile(offers[:,None],len(j_star_idx )), ~offering_ind)
        best_offers_id = np.argmax(masked_matrix, axis = 0)
  
        ### Update value and matching
        V_j[j_star_idx] += np.maximum(offers[best_offers_id] - V_j[j_star_idx] , tol) #j_assigned[j_star_idx] * self.tol
        mu_ij[:,j_star_idx] = 0
        mu_ij[i_offering[best_offers_id],j_star_idx] = 1

        unassigned = mu_ij.sum(1) == 0
        n_unassigned =unassigned.sum()
        iter += 1
   
    id_i , id_j  =  np.where(mu_ij[:,:-1] == 1)
    
    U_i = np.ones(self.n) * self.lb_U
    U_i[id_i] = self.get_U_ij(V_j, id_i)[np.arange(len(id_i)), id_j]    
   
    return mu_ij, U_i, V_j


OneToOneITU.ITU_auction = ITU_auction

## Examples


In [63]:
n_small = 10
m_small = 10
example_small = OneToOneITU( n = n_small,m = m_small, size_params= (2,5) , random_seed= 778, lbs =(0,0), tol = 1e-10)

In [64]:
example_small.A_ij 

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [67]:
mu_small , U_small , V_small = example_small.ITU_auction(40000, 1e-10)

In [70]:
example_small.check_all((mu_small[:,:-1] , U_small , V_small ))

____
<h1>Feasibility</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

#matched: 9 over 10

____
<h1>Generalized Complementary Slackness</h1>

...

____
<h1>Individual Rationality</h1>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>